In [1]:
import pandas as pd

import ee
from functools import partial
from traitlets import (
    Dict, HasTraits, Unicode, Any, link, observe
)
import ipyvuetify as v

import sepal_ui.sepalwidgets as sw
from sepal_ui.scripts import utils as su
from component.widget.custom_widgets import *
from component.scripts import *
from component.tile import *
from component.message import cm
from component.frontend.styles import *

Styles()

ResizeTrigger()

Styles()

In [2]:
ee.Initialize()

In [3]:
class GeeSelector(v.Card):
    
    asset = Any('').tag(sync=True)
    code_col = Any('').tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        
        self.class_ = 'pa-4'
        title = v.CardTitle(children=[cm.ui.title])
        description = v.CardText(class_='py-0', children=[sw.Markdown(cm.ui.description)])
        
        super().__init__(*args, **kwargs)
        
        self.alert_output = Output()
        self.alert = sw.Alert()
        self.alert_dialog = Dialog(output=self.alert_output)
        
        self.ee_asset = None
        self.w_mapper = Remap(
            dense=True, 
            max_height=400,
            _metadata={'name':'mapper'}
        )
        
        self.asset_selector = AssetSelect(
            label=cm.remap.label, 
            default_asset = 'users/dafguerrerom/FAO/LULC_2012_AOI'
        )
        
        self.w_code = v.Select(label='', class_='pr-4', v_model='')
        self.mapper_btn = sw.Btn(cm.remap.btn, small=True)
        
        self.w_asset = Flex(
            _metadata = {'name':'code'},
            class_='d-flex align-center mb-2',
            children=[self.w_code, self.mapper_btn]
        )
        
        # List of components whose could be hidden/showed
        
        self.components = {
            'mapper': self.w_mapper,
            'code' : self.w_asset
        }
        
        # Define view
        self.children = [
            self.alert_dialog,
            title,
            description,
            self.asset_selector,
        ]
        
        #Link traits
        link((self, 'asset'), (self.asset_selector, 'v_model'))
        link((self.w_code, 'v_model'), (self, 'code_col'))
        
        # Create Events
        self.mapper_btn.on_event('click', self._get_mapper_matrix_event)
        
        # Decorate functions
        self._get_mapper_matrix = loading(self.alert, self.alert_dialog, self.mapper_btn)(self._get_mapper_matrix)
        self._validate_asset = loading(self.alert, self.alert_dialog)(self._validate_asset)
        
        
    def _get_mapper_matrix_event(self, widget, event, data):
        self._get_mapper_matrix()
        
    def _get_mapper_matrix(self):
        
        assert (self.code_col != ''), cm.remap.error.no_code_col
        
        if self.asset_type == 'TABLE':
            code_fields = self._get_fields()
        elif self.asset_type == 'IMAGE':
            code_fields = self._get_classes()
        
        # Create mapper widget
        self.w_mapper._get_matrix(code_fields)

        self._add_widget(self.w_mapper, 'mapper')

    def _hide_components(self):
        """Hide all possible componentes"""
        self.code_col = ''
        for component in self.components.values():
            su.hide_component(component)
    
    def _validate_asset(self, asset):
        
        asset_info = ee.data.getAsset(asset)
        self.asset_type = asset_info['type']
        
        if self.asset_type == 'TABLE':
            self.w_code.label = cm.remap.code_label
            self.ee_asset = ee.FeatureCollection(asset)

        elif self.asset_type == 'IMAGE':
            self.w_code.label = cm.remap.band_label
            self.ee_asset = ee.Image(asset)
        else:
            err_str = cm.remap.error_type
            raise AttributeError(err_str)

    @observe('asset')
    def _get_items(self, change):
        
        asset = change['new']
        
        if asset:
            
            # Hide previous loaded components
            self._hide_components()
            self.w_code.items=[]
            self._add_widget(self.w_asset, 'code')
            self._validate_asset(asset)

            self.w_code.loading=True

            # Get columns of dataset
            if self.asset_type == 'TABLE':
                columns = self._get_cols()

            elif self.asset_type == 'IMAGE':
                columns = self._get_bands()

            # Fill widgets with column names
            self.w_code.items = columns

            self.w_code.loading=False
    
        
    def _add_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        su.show_component(widget)
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name not in widgets_name:
            self.children = self.children + [widget]
            
    def _remove_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name in widgets_name:
            
            children = self.children
            children.remove(widget)
            
            self.children = children

    def _get_bands(self):
        """Get bands from Image asset"""
        return list(self.ee_asset.bandTypes().getInfo().keys())
        
    def _get_fields(self):
        """Get fields from Feature Collection"""
        return sorted(list(set(self.ee_asset.aggregate_array(self.code_col).getInfo())))

    def _get_cols(self):
        
        if self.ee_asset:
            columns = ee.Feature(self.ee_asset.first()).propertyNames().getInfo()
            return sorted([str(col) for col in columns if col not in ['system:index', 'Shape_Area']])
        
    def _get_classes(self):
        """Get raster classes"""
        
        # Reduce image
        reduced = self.ee_asset.reduceRegion(
          reducer = ee.Reducer.autoHistogram(maxBuckets=40000), 
          geometry = self.ee_asset.geometry(), 
          scale=30, 
          maxPixels=1e13
        )

        array = ee.Array(ee.List(reduced.get(self.code_col))).getInfo()
        df = pd.DataFrame(data=array, columns=['code', 'count'])
        
        return list(df[df['count']>0]['code'].unique())
        

In [4]:
gee_selector = GeeSelector()

In [5]:
gee_selector

GeeSelector(children=[Dialog(children=[Card(children=[Output()])], max_width=436.0, overlay_color='black', v_m…

In [ ]:
gee_selector.w_mapper.matrix